## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:

- [Getting Started](#Getting-Started)
    - [initialization](#initialization)
    - [show available hardware info list](#show-available-hardware-information)
    - [choose hardware config](#Set-default-hardware-config-(Optional))
- [Creating And Deploying Task](#Creating-And-Deploying-Task)
    - [estimate payment amount](#estimate-payment-amount-optional)
    - [Auto Pay Path](#Auto-Pay-Path)
        - [create task with auto_pay](#Create-task-(auto_pay))
        - [renew task with auto-pay](#renew-task-auto-pay-optional)
    - [No Auto Pay Path](#No-Auto-Pay-Path)
        - [create task with no auto_pay to get `task_uuid`](#Create-task-(No-auto_pay))
        - [make payment](#make-payment-(Optional))
        - [validate payment](#validate-payment-to-deploy-task)
        - [renew task](#renew-task-(no-auto-pay)-optional)
    - [terminate task](#terminate-task-optional)
    - [claim review](#claim-review-optional)
- [View Deployed Task Results](#View-Deployed-Task-Results)
    - [follow up task status](#follow-up-task-status-optional)
    - [show result](#show-result)


# Getting Started


### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

wallet_address = os.getenv('WALLET')
private_key = os.getenv('PK')
api_key = os.getenv('API_KEY')
swan_orchestrator = swan.resource(api_key=api_key, service_name='Orchestrator')

In [2]:
r = swan_orchestrator.contract_info
print(json.dumps(r, indent=2))

{
  "client_contract_address": "0x20a67c6Bea000fAf0BE862BB254F092abF0E5b98",
  "payment_contract_address": "0x5094A609Af5184d076Be2DF741820732126b4Fd2",
  "rpc_url": "https://rpc-atom-internal.swanchain.io",
  "swan_token_contract_address": "0x91B25A65b295F0405552A4bbB77879ab5e38166c"
}


### Show available hardware information

In [3]:
hardwares_info = swan_orchestrator.get_hardware_config()
print(hardwares_info)
hardware_id = 0 # 'C1ae.medium'
region = 'global'

[{'id': 0, 'name': 'C1ae.small', 'description': 'CPU only · 2 vCPU · 2 GiB', 'type': 'CPU', 'region': ['Virginia-US', 'Quebec-CA', 'Kwai Tsing-HK', 'Guangdong-CN', 'Utah-US', 'North Holland-NL', 'Washington-US', 'Bavaria-DE', 'North West-SG', 'Kowloon-HK', 'Ōsaka-JP', 'Texas-US', 'Tokyo-JP', 'Sichuan-CN', 'Missouri-US', 'Central Singapore-SG', 'Saxony-DE', 'Central and Western-HK', 'Bashkortostan Republic-RU', 'Kowloon City-HK', 'North Rhine-Westphalia-DE', 'Kuala Lumpur-MY', 'Jakarta-ID', 'North Carolina-US', 'Metro Manila-PH', 'Bangkok-TH', 'Seoul-KR'], 'price': '0.0', 'status': 'available'}, {'id': 1, 'name': 'C1ae.medium', 'description': 'CPU only · 4 vCPU · 4 GiB', 'type': 'CPU', 'region': ['Virginia-US', 'Quebec-CA', 'Kwai Tsing-HK', 'Guangdong-CN', 'North Holland-NL', 'North West-SG', 'Kowloon-HK', 'Ōsaka-JP', 'Texas-US', 'Washington-US', 'Tokyo-JP', 'Sichuan-CN', 'Missouri-US', 'Central Singapore-SG', 'Saxony-DE', 'Central and Western-HK', 'Bashkortostan Republic-RU', 'Kowloon 

### Set default hardware config (Optional)

choose a default hardware with its hardware id and region. Set default `hardware_id` and `region` will be used in the steps to deploy task if not provided as parameters in future functions

In [4]:
hardware_id = 0 # 'C1ae.medium'
region = 'global'
if swan_orchestrator.set_default_task_config(hardware_id, region):
    print("Successfully set up default task configuration")

Successfully set up default task configuration


# Creating And Deploying Task

### Estimate Payment amount (Optional)

Estimate the cost of deploying the task on specified hardware for duration (in seconds)

In [5]:
duration = 3600 # 1 hour or 3600 seconds

amount = swan_orchestrator.estimate_payment(
    duration=duration, # Optional: Defaults to 3600 seconds or 1 hour
    hardware_id=hardware_id, # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

print(amount)

0


## Auto Pay Path

### Create task (auto_pay)

This step creates, pays and deploys task. It also gets `task_uuid`, which is useful in task info steps.

`wallet_address` is mandatory. `job_source_uri` is mandatory in this demo, please check out example-demo-prebuilt-image if deploying prebuilt images with `image_name`.

In this section, `private_key` is mandatory. If you do not want to use `private_key` and pay through SDK, please check out [create task with no auto_pay](#Create-task-(No-auto_pay))

`job_source_uri` is repository url of code to be deployed, must contain a must contain a dockerfile or deploy.yaml file

In [6]:
# job_source_uri = '<github repository url of code to be deployed>'

# Demo example: uncomment to use
job_source_uri = 'https://github.com/alphaflows/tetris-docker-image.git'

result = swan_orchestrator.create_task(
    wallet_address=wallet_address,
    job_source_uri=job_source_uri,
    auto_pay=True, # Optional: Defaults to false, but in this section's path, set to True
    private_key=private_key,
    hardware_id=0, # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
    region='global', # Optional: Defaults to region set in set_default_task_config or global if not set
    duration=duration, # Optional: Defaults to 3600 seconds
)

print(json.dumps(result, indent=2))

# Store the task_uuid of created task
task_uuid = result['id']

{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719612537,
      "end_at": 1719616137,
      "id": 26645,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x662aC7c5BA4193744224ee507298b4e5f92D6fDd",
      "source": "v2",
      "start_at": 1719612537,
      "start_in": 300,
      "status": "created",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719612537,
        "duration": 3600,
        "end_at": 1719616137,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://data.mcs.lagrangedao.org/ipfs/QmacTTg9nYycQKHov2AgnLhqFEErT4wZB1pmEWfYowxYvm",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "region": "global",
          "vcpu": "2"
        },
        "start_at": 1719612537,
        "status": "paid",
        "storage_source": "

### renew task (auto-pay) (optional)

Extend `task_uuid` by `duration`. Using auto pay automatically makes a transaction to SWAN contract and extends the task.

In [7]:
renew_task = swan_orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    auto_pay=True, # Optional: Defaults to False, in this demo path set to True
    private_key=private_key,
    hardware_id=hardware_id # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed task")

successfully renewed task


If completed steps above, please go to [View Deployed Task Results](#View-Deployed-Task-Results) to view results, or [terminate task](#terminate-task) for more functions related to tasks

## No Auto Pay Path

### Create task (No auto_pay)

This creates task, but does not pay or deploy task. It gets `task_uuid`, which is useful in payment steps.

`wallet_address` is mandatory. `job_source_uri` is mandatory in this demo, please check out example-demo-prebuilt-image if deploying prebuilt images.

`job_source_uri` is repository url of code to be deployed, must contain a must contain a dockerfile or deploy.yaml file

In [ ]:
# job_source_uri = '<github repository url of code to be deployed>'

# Demo example: uncomment to use
job_source_uri = 'https://github.com/alphaflows/tetris-docker-image.git'

result = swan_orchestrator.create_task(
    wallet_address=wallet_address,
    job_source_uri=job_source_uri,
    hardware_id=hardware_id, # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
    region='global', # Optional: Defaults to region set in set_default_task_config or global if not set
    duration=duration, # Optional: Defaults to 3600 seconds
    auto_pay=False, # Optional: Defaults to false
)

print(json.dumps(result, indent=2))

# Store the task_uuid of created task
task_uuid = result['id']

### Make Payment (Optional)

This step is using `task_uuid`, `private_key`,  `duration`, and `hardware_id` to submit payment and make task eligible for assigning if payment successful via swan SDK. `task_uuid`, `private_key` are mandatory.

If following this section, please skip to - [View Deployed Task Results](#View-Deployed-Task-Results) next.
If do not want to submit payment with swan SDK, and directly made payment to swan contract instead, please move onto next section.

In [ ]:
if result_validation := swan_orchestrator.make_payment(
    task_uuid=task_uuid, 
    private_key=private_key,
    duration=3600, # Optional: Defaults to 3600 seconds (1 hour)
    hardware_id=0 # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

### Validate Payment to deploy task

Only use this if paid directly to contract. If used make_payment section above, ignore this section.

This step will use `tx_hash` and `task_uuid` to validate the payment and then make task eligible for assigning if validation successful

Will Delete the submit_payment code block below in final version

In [ ]:
if result_validation := swan_orchestrator.validate_payment(
    tx_hash="<tx_hash of payment of task_uuid to swan contract>",
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

### Renew Task (no auto-pay) (Optional)

Extend `task_uuid` by `duration`. `tx_hash` of payment for extension required.

Will Delete the submit_payment code block below in final version

In [ ]:
renew_task = swan_orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=60, # Optional: Defaults to 3600 seconds (1 hour)
    tx_hash="<tx_hash of payment of task_uuid to swan contract>", # tx_hash of payment to swan contract for this task
    hardware_id=hardware_id # Optional: Defaults to hardware_id set in set_default_task_config or 0 (free) if not set
)

if renew_task and renew_task['status'] == 'success':
    print(f"successfully renewed {task_uuid}")
else:
    print(f"Unable to renew {task_uuid}")

### terminate task (Optional)

Terminate the task `task_uuid` and get a refund for remaining time

In [ ]:
terminate_status = swan_orchestrator.terminate_task(task_uuid)
if terminate_status['status'] == 'success':
    print(f"Terminated {task_uuid} successfully")
else:
    print(f"Failed to terminate {task_uuid}")

### claim review (Optional)

Review if `task_uui` uptime is above 90% and give refund if below 90%

In [ ]:
claim_review = swan_orchestrator.claim_review(task_uuid)
print(claim_review)

## View Deployed Task Results

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [8]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "beneficiary": "0x2c03538AEeD876198d8f376F3556857d1B118AbC",
        "cp_account_address": "0x3eFb0f95bB5a1Eb3c9868e9c535C6aBf19dD2876",
        "created_at": 1718266434,
        "id": 66,
        "lat": 22.3193,
        "lon": 114.169,
        "multi_address": [
          "/ip4/103.201.24.47/tcp/8091"
        ],
        "node_id": "04e0961444d2f1856f6ebccf3b02f855bde584a9a5bb72fa60ab5ec8418a0a200236d2673acd025093eba0fb8f7c7ebbaceab433b0343181eca93d5e8d7d7466d9",
        "online": 1,
        "owner_address": "0x2c03538AEeD876198d8f376F3556857d1B118AbC",
        "region": "Kowloon-HK",
        "task_types": "[1, 2, 3, 4]",
        "updated_at": 1719612153,
        "version": "2.0",
        "worker_address": "0x2c03538AEeD876198d8f376F3556857d1B118AbC"
      },
      {
        "beneficiary": "0x8c68475E62B21d42B8a35206658F2B0f0a165174",
        "cp_account_address": "0x49992952342e848d2048B09b85E877e6d39Ee869",
        "created_a

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [9]:
result_url = swan_orchestrator.get_real_url(task_uuid)
print(result_url)

['https://a4pvvz4psu.zzz.storefrontiers.cn', 'https://f08bx9434r.tiaokong.top', 'https://rmu8z4fnbq.']


Check the response codes of the result_url

In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

for url in result_url:
    response = requests.get(url, headers=headers)

    try:
        print(json.dumps(response.json(), indent=4))
    except Exception as e:
        print(e)
        print(response)